# Working with Pandas DataFrames - Part 2

Last session we introduced Pandas. Now we review the basics of using Pandas, with a more interactive workflow using relevant data to solidify understanding and facility with this powerful Python toolkit.

## Reviewing Pandas Basics

In [ ]:
from pandas import DataFrame
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
df = pd.read_csv('data/ca_tracts_pop_cleaned.csv')

### A bit more practice with indexing in Pandas

The first use of indexing is to use a slice, just like we have done with other Python objects. Below we slice the first 5 index values of the first dimension of the dataframe.

In [ ]:
df[:5]

The first indexing method is equivalent to usinf the iloc indexing method, which uses the integer based indexing, purely based on the location of the index.

In [ ]:
df.iloc[:5]

By the way, Pandas has a built-in short-cut method called 'head' that shows the first 5 rows of a dataframe.  Very handy when the dataframe is large to just look at a few rows.

In [ ]:
df.head()

A second way to index is using loc, which uses the labels of the index. Note that this approach includes the second value in the index range, whereas iloc does not.

In [ ]:
df.loc[:5]

Note that indexing can work for both rows and colums

In [ ]:
df.loc[:5, : 'GEOID']

In [ ]:
df.iloc[:5, :4]

We can select rows based on their value as well.  Notice that we nest df[df[condition]] to get this result.

In [ ]:
df[df['Population'] < 200]

Below we find and print records that are in El Dorado County, using the str attribute and 'contains' to search for the county name in geodisplay.

In [ ]:
df[df['geodisplay'].str.contains('El Dorado County')]

In [ ]:
df[df['GEOID']==6017031900]

Here we show how to set a value of a cell in the table, identifying a specific row by index label, and setting its population, in this case to a None value, which Pandas interprets as a NaN (missing value).

In [ ]:
df.loc[688,'Population'] = None

We can filter for values that are Null

In [ ]:
df[df['Population'].isnull()]

Or more commonly, filter out the null values.

In [ ]:
df[df['Population'].notnull()]

We can also fill the missing values with a specific number. In this case let's just put the value back that we overwrote earlier with a NaN:

In [ ]:
df.loc[688,'Population']  = 133

In [ ]:
df.loc[688]

The more common use case for this approach would be to replace special values like -9999 with Nan, or vice versa.

### String Manipulation in DataFrames

We saw last time how to use the str attribute to do a bit of string manipulation. Below let's create a new column, called 'state', populated by getting the last element in the geodisplay field.

In [ ]:
df['state'] = df['geodisplay'].str.split(',').str[2]
df[:5]

We use the same approach to add a 'county' column to our dataframe, pulling the values from the geodisplay column.

In [ ]:
df['county'] = df['geodisplay'].str.split(',').str[1].str.replace(' County', '')
df[:5]

Here is a different way to get the County values extrated.

In [ ]:
df['county'] = df['geodisplay'].str.split(',').str[1].str.split(' ').str[1:-1].str.join(' ')
df

Let's test to make sure our approach preserves two word County names..

In [ ]:
df[df['geodisplay'].str.contains('El Dorado County')]

Let's see what the full list of unique county names is...

In [ ]:
df['county'].unique()

Check-in time: How would you count how many census tracts are in each county? http://bitly.com/cp255

Now let's create a census tract column.

In [ ]:
df['Tract'] = df['geodisplay'].str.split(',').str[0].str.split().str[2]
df


Could you extract Tract from the GEOIDLONG column instead?

In [ ]:
df['Tract'] = df['GEOIDLONG'].str[-6:-2] + '.' + df['GEOIDLONG'].str[-2:]
df.head()

## Merging / Joining Dataframes

Pandas has a very powerful set of methods to merge dataframes. Let's review some on a simple example)

In [ ]:
favorite_numbers = pd.DataFrame(
    [["Paul", 42],
     ["Paul", 3.14],
     ["Arezoo", 7],
     ["Arezoo", 9],
     ["Sam", 3],
     ["Geoff", np.NaN]], columns = ["Name", "Number"])
email_addr = pd.DataFrame(
    [["Paul", "waddell@berkeley.edu"],
     ["Arezoo", "arezoo.bz@berkeley.edu"],
     ["Sam", "maurer@berkeley.edu"],
     ["Geoff", "gboeing@berkeley.edu"],
     ["Max", "magardner@berkeley.edu"]], columns = ["Name", "Email"])

The first table contains favorite numbers of some famous people.  (The numbers have been changed to protect their identity.)

In [ ]:
favorite_numbers

The second table contains the individuals email addresses.

In [ ]:
email_addr

## Many ways to join the data

There are actually many ways you could imagine combining data from both of these tables.  In the following we work through a few example methods.

### Merge

Probably the most general and standard way to join tables in pandas is to use the merge function:

In [ ]:
pd.merge(favorite_numbers, email_addr)

Notice in the above join that:
1. The **`Name`** column is used to define which records match from each table.  Pandas will by default join on any matching column names.
1. Only the records that occur in both tables are included in the final table.  This is called an inner join.
1. Joey occurs 4 times since the name Joey had two email addresses and two favorite numbers.

Many of the joins you will do in data science will either be inner joins or left joins (see below). 

We could be more specific about the join using the following additional arguments.

In [ ]:
pd.merge(favorite_numbers, email_addr, on="Name", how="inner")

#### Left Merge

A left join will keep all the entries in the left table even if they have no matching entry in the right table.  For example we didn't have Nhi or Bob's email address and so they appear as missing values rather than being dropped from the join.

In [ ]:
pd.merge(favorite_numbers, email_addr, on="Name",how="left")

#### Outer Merge

The outer join keeps entries in both tables even if they don't have a match in the other and substitutes NaN for missing values. 

pd.merge(favorite_numbers, email_addr, on="Name", how="outer")

### Join

Pandas also provides a join function which joins two tables on their index.  This function also let's you specify what kind of join you would like.

In [ ]:
favorite_numbers.set_index("Name").join(email_addr.set_index("Name"), how="inner")

### More resources to learn about merges and joins

These are powerful tools and there are some subtleties in using them.  I recommend doing a fair amount of reading and a lot of practicing to get these ideas and the accompanying syntax internalized into your programming toolkit.

Here is another good (and short) tutorial: https://chrisalbon.com/python/pandas_join_merge_dataframe.html

And as always, **ReadTheDocs**: http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging

## Using Merge on Real Datasets and Using String Operations

Now that we have some concepts and syntax under control, let's try using them on some larger data tables from the Census, and learn how to use string operations to work with these data.

### County level Census Data for the U.S.

Lets say we want to analyze county level population trends. I downloaded population two estimates tables for U.S. Counties: one contains 2000-2010, and the other has 2010-2016.  Lets load them and do some data exploration and manipulation to get a merged county level population series for the 2000 - 2016 period.

One thing that you will encounter as you get csv files from various locations, is that the character encoding might be unusual, and require setting the encoding on the read statement.  Full documentation on this is available here:
https://docs.python.org/3/library/codecs.html#standard-encodings

Some of the most common ones are:

Latin1
iso-8859-1
utf_8
utf_16
utf_32

Let's see what happens when you try loading a csv file with an encoding issue:

In [ ]:
co00 = pd.read_csv('data/co-est00int-tot.csv')
co00.head()

In [ ]:
co00 = pd.read_csv('data/co-est00int-tot.csv', encoding='Latin1')
co00.head()

In [ ]:
co16 = pd.read_csv('data/co-est2016-alldata.csv', encoding='latin1')
co16.head()

**116 columns!?!  We don't want all of those!  How can we keep just the columns we want? The ones containing POPESTIMATE, and the initial geographic columns?**

In [ ]:
co16s = co16.loc[:,:'POPESTIMATE2016']
co16s.columns

Ahhh, that's better...

In [ ]:
co16s.head()

**But wait... what's going on here?  There seem to be at least two SUMLEV values and it looks suspiciously like there are State level summaries of POPESTIMATES embedded in this County-level file.  Let's check how many records there are for each SUMLEV in the file...**

In [ ]:
co16s['SUMLEV'].value_counts()

Hmm, OK, so we have 51 SUMLEV 40, which seem to be States, plus one... let's pull those into a new dataframe.

In [ ]:
st00 = co00[co00['SUMLEV']==40]
st00.head()

In [ ]:
st00.shape

In [ ]:
st00['STNAME'].unique()

So what is that 51st entry in the set of States?

Now let's pull the state records out of the 2016 file.

In [ ]:
st16 = co16s[co16s['SUMLEV']==40]
st16.head()

**Now how can we merge the 2000 and 2016 state dataframes?**

In [ ]:
stjoin = pd.merge(st00,st16, on='STATE')
stjoin.head()

**Looks like it worked!  But what are all those columns with _x and _y suffixes?**

Seems like a mess to keep all those duplicate columns and have them get renamed like this... what to do...?

Maybe we could find the columns that are different in the second dataframe and just add those to the first dataframe?

In [ ]:
cols_to_use = list(st16.columns.difference(st00.columns))
cols_to_use

Would it work if we tried to join these two using this as the list of columns from ST16?  What would be the join column? Seems like that would be pretty handy to have...

In [ ]:
cols_to_use.append('STATE')
cols_to_use

Now we can do a cleaner merge using STATE:

In [ ]:
stjoin2 = pd.merge(st00, st16[cols_to_use], on='STATE')
stjoin2.head()

Looking pretty good.  But there are still a lot of columns we really don't want.  Let's drop those, and set the index to STNAME.

In [ ]:
stjoin3 = stjoin2.drop(['SUMLEV','STATE','REGION','DIVISION','COUNTY','ESTIMATESBASE2000','ESTIMATESBASE2010','CENSUS2010POP','CTYNAME'], axis=1)
stjoin3.set_index('STNAME')

In [ ]:
stjoin3.columns

Much nicer.  But I really don't like those column names.  Can't we just make them simple years?

In [ ]:
stjoin4 = stjoin3
stjoin4.columns = list(stjoin3.columns[0:1])+list(range(2000,2017))
stjoin4 = stjoin4.set_index('STNAME')
stjoin4.head()

It might also be handy to transpose the data so that the rows become columns and vice versa.  Let's do that and select just a couple of states.

In [ ]:
ST = stjoin4.transpose()
ST[['California','New York']]

Maybe a quick plot of the population trends in these states?

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(ST[['California', 'Texas']])

And for those of you who love your Excel for making charts, since mine is pretty ugly, go for it:

In [ ]:
writer = pd.ExcelWriter('data/state_population.xlsx')
stjoin4.to_excel(writer,'Sheet1')
writer.save()

## Data Exploration and Cleaning

**Let's load some crime call events from the Berkeley Police Department and explore those data...  this is an example of having to wrangle some messy, real world data.  It's advanced work that you now can handle in Python!**

For example lets say we want to find out how many vandalism calls there are each day of the week. Let's load some data and figure out how to answer that.

https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5

In [ ]:
calls = pd.read_csv('data/Berkeley_PD_-_Calls_for_Service.csv')
calls.head()

Let's also load a lookup for CVDOW (Day of Week) to the day names

In [ ]:
cvdow = pd.read_csv("data/cvdow.csv")
cvdow

How many calls are in this dataset?

In [ ]:
len(calls)

### Preliminary observations on the data?

1. `EVENTDT` -- Contain the incorrect time stamp (all the times are 12:00 am)
1. `EVENTTM` -- Contains the time in 24 hour format (What timezone?)
1. `InDbDate` -- Appears to be correctly formatted and appears awfully consistent in time.
1. **`Block_Location` -- Errr, what a mess!  newline characters, and Geocoordinates all merged!!  Fortunately, this field was "quoted" otherwise we would have had trouble parsing the file. (why?)**
1. `BLKADDR` -- This appears to be the address in Block Location.
1. `City` and `State` seem redundant given this is supposed to be the city of Berkeley dataset.

In [ ]:
new = pd.merge(calls,cvdow, on='CVDOW', how='inner')
new.head()

How could we find out how many Vandalism calls happen by Day of the week?

In [ ]:
new['Day'][new['OFFENSE']=='VANDALISM'].value_counts()

In [ ]:
new['Block_Location'].head()

In [ ]:
new['Block_Location'][0]

In [ ]:
new['Block_Location'].str.find('(').head()

In [ ]:
for label in new.Block_Location.index:
    start=new.Block_Location[label].find('(') + 1
    end= start+9
    new.loc[label,'Lat'] = new.Block_Location[label][start:end].strip(',')

In [ ]:
new.head()

OK, now do this to add a Long column to the dataframe...

In [ ]:
for label in new.Block_Location.index:
    start=new.Block_Location[label].find('-')
    end= new.Block_Location[label].find(')')
    new.loc[label,'Lon'] = new.Block_Location[label][start:end].strip(',')

In [ ]:
new.head()

How could you check whether city and state are all redundant (since this is supposed to be City of Berkeley data)?

In [ ]:
new['City'].value_counts()

In [ ]:
new['State'].value_counts()

If they are duplicated, delete those columns

In [ ]:
new.drop(['State', 'City'], axis=1)

Let's fix the date time to be able to parse the dates and work with them. Pandas has very helpful functionality using a datetime datatype.

In [ ]:
#new['EVENTDT'].to_datetime()
new['DATE'] = pd.to_datetime(new['EVENTDT'])
new['MONTH'] = new['DATE'].dt.month

In [ ]:
new.head()